# Group Surveys Feature Determination

## Preparation

### Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

## Function Definition

### Read and Concatenate Tables

In [ ]:
def read_collapse_tables(whats_features, meetings_features, surveys_features, grades):

  df_whats= pd.read_excel(whats_features, index_col='group')
  df_meetings= pd.read_excel(meetings_features,header=0,skiprows=range(1, 5)).rename_axis('Id')
  df_meetings.index += 1
  df_meetings= df_meetings.drop('group_name', axis=1)
  df_surveys= pd.read_excel(surveys_features, index_col='group')
  df_grades= pd.read_excel(grades, index_col='Id').rename_axis('new_index_col')
  df_grades= df_grades.drop('Group', axis=1)

  df = pd.concat([df_meetings, df_whats, df_surveys, df_grades], axis=1)
  df = df.dropna(axis=1)

  return df

### Apply Different Scalings

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def process_scalings(df, scaler):
    if scaler == 'standard':
        scaler_obj = StandardScaler()
    elif scaler == 'minmax':
        scaler_obj = MinMaxScaler()
    elif scaler == 'robust':
        scaler_obj = RobustScaler()
    elif scaler == 'quartile':
        scaler_obj = QuantileTransformer(output_distribution='uniform', n_quantiles= len(df))
    else:
        raise ValueError("Invalid scaler. Please choose 'standard', 'minmax', 'robust', or 'quartile'.")

    scaled_df = df.copy()  # Create a copy of the original DataFrame

    # Scale the numerical columns
    numeric_cols = scaled_df.select_dtypes(include=['float64', 'int64']).columns
    scaled_df[numeric_cols] = scaler_obj.fit_transform(scaled_df[numeric_cols])

    return scaled_df


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
import pandas as pd
import numpy as np

def process_scalings(df, scaler):
    if scaler == 'standard':
        scaler_obj = StandardScaler()
    elif scaler == 'minmax':
        scaler_obj = MinMaxScaler()
    elif scaler == 'robust':
        scaler_obj = RobustScaler()
    elif scaler == 'quartile':
        scaler_obj = QuantileTransformer(output_distribution='uniform', n_quantiles= len(df))
    else:
        raise ValueError("Invalid scaler. Please choose 'standard', 'minmax', 'robust', or 'quartile'.")

    scaled_df = df.copy()  # Create a copy of the original DataFrame

    # Filter and scale only the numerical columns
    numeric_cols = scaled_df.select_dtypes(include=['float64', 'int64']).columns
    scaled_df[numeric_cols] = scaler_obj.fit_transform(scaled_df[numeric_cols])

    return scaled_df

### Min-Max Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scaling_df(df):
    scaler = MinMaxScaler()
    scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return scaled_df


### Display Correlation Matrix

In [ ]:
def corr_matrix(df):
  correlation_matrix = df.corr()

  # Create a figure and axes
  fig, ax = plt.subplots(figsize=(10, 8))

  # Create the heatmap using seaborn
  #sns.heatmap(correlation_matrix, cmap='coolwarm', ax=ax)

  # Set the title and axis labels
  ax.set_title('Correlation Matrix')
  ax.set_xlabel('Variables')
  ax.set_ylabel('Variables')

  # Rotate the x-axis labels for better readability
  plt.xticks(rotation=90)

  # Display the heatmap
  plt.show()

### Display Scatter Plot

In [ ]:
def scatterplot (df,x_column,y_column):
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  import numpy as np
  from sklearn.linear_model import LinearRegression
  from sklearn.preprocessing import PolynomialFeatures

  # Assuming your DataFrame is called 'df' and you want to check relationships between columns 'x' and 'y'
  x = df[x_column]
  y = df[y_column]

  # Create a scatter plot to visualize the relationship
  plt.scatter(x, y)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title('Scatter Plot')
  plt.show()

  # Fit a polynomial regression model
  degree = 2  # Degree of the polynomial
  poly_features = PolynomialFeatures(degree=degree)
  X_poly = poly_features.fit_transform(x.values.reshape(-1, 1))

  model = LinearRegression()
  model.fit(X_poly, y)

  # Calculate predicted values
  y_pred = model.predict(X_poly)

  # Plot the fitted curve
  plt.scatter(x, y)
  plt.plot(x, y_pred, color='red', label=f'Degree {degree} Polynomial Fit')
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title('Polynomial Regression')
  plt.legend()
  plt.show()


### Correlation Significance

In [ ]:
def check_correlation_significance(df, col1, col2):
  import scipy.stats as stats

  # Assuming your DataFrame is called 'df'

  # Extract the 'performance' and 'recreational_perceived_mean' columns
  performance = df[col1]
  recreational_mean = df[col2]

  # Perform the hypothesis test
  corr_coeff, p_value = stats.pearsonr(performance, recreational_mean)

  # Print the results
  print("Correlation coefficient:", corr_coeff)
  print("p-value:", p_value)

  # Compare the p-value to the significance level
  alpha = 0.05  # Significance level
  if p_value < alpha:
      print("The correlation is statistically significant (reject H0).")
  else:
      print("The correlation is not statistically significant (fail to reject H0).")


### Apply Scalings given DF and print correlation

In [ ]:
def apply_scalings(df, column):

  correlations=[]
  scalers= ['minmax','quartile']

  for scaler in scalers:

    df= process_scalings(df, scaler)

    # Calculate the correlations
    corr1 = df.corrwith(df[column])
    #filtered_series = corr1[abs(corr1) > 0.6]
    correlations.append(corr1)

  df_final = pd.concat(correlations, axis=1)
  df_final.columns = scalers

  #fileterd_df = df_final[(np.abs(df) > 0.6).any(axis=1)]

  return df_final


## Use of Function

In [ ]:
grades = r'/content/drive/MyDrive/Projects/tps/grades/data/2_group_grades.xlsx'
whats_features= r'/content/drive/MyDrive/Projects/tps/whatsapp/data/2_whatsapp_group_features.xlsx'
meetings_features= r'/content/drive/MyDrive/Projects/tps/meetings/data/12. features/7_group_meetings_features_final.xlsx'
surveys_features=  r'/content/drive/MyDrive/Projects/tps/surveys/data/2_happimeter_group_surveys.xlsx'
out_file= r'/content/drive/MyDrive/Projects/tps/finals/data/1_group_features.xlsx'

In [ ]:
df = read_collapse_tables(whats_features, meetings_features, surveys_features, grades)
df.rename_axis('Id', inplace=True)
#df = min_max_scaling_df(df)

In [ ]:
df.head(12)

,num_members,num_recordings,total_duration_recorded,share_screen_time,members_show_face_percen,screen_share_percen,total_duration_spoken,ratio_spoken_recorded,avg_meeting_turn_duration,total_turns,turns_in_a_min,average_meeting_words_message,total_words,words_in_a_min,avg_indiv_spoken_time,stdev_indiv_spoken_time,min_indiv_spoken_time,max_indiv_spoken_time,diff_indiv_spoken_time,avg_indiv_spoken_time_ratio,stdev_indiv_spoken_time_ratio,min_indiv_spoken_time_ratio,max_indiv_spoken_time_ratio,diff_indiv_spoken_time_ratio,avg_average_turn_duration,stdev_average_turn_duration,min_average_turn_duration,max_average_turn_duration,diff_average_turn_duration,avg_average_turn_duration_ratio,stdev_average_turn_duration_ratio,min_average_turn_duration_ratio,max_average_turn_duration_ratio,diff_average_turn_duration_ratio,avg_avg_time_without_speaking,stdev_avg_time_without_speaking,min_avg_time_without_speaking,max_avg_time_without_speaking,diff_avg_time_without_speaking,avg_avg_time_without_speaking_ratio,stdev_avg_time_without_speaking_ratio,min_avg_time_without_speaking_ratio,max_avg_time_without_speaking_ratio,diff_avg_time_without_speaking_ratio,avg_max_time_without_speaking,stdev_max_time_without_speaking,min_max_time_without_speaking,max_max_time_without_speaking,diff_max_time_without_speaking,avg_max_time_without_speaking_ratio,...,authority_respect_score_avg_difference,purity_sanctity_score_mean,purity_sanctity_score_std,purity_sanctity_score_avg_difference,dummy_question1_mean,dummy_question1_std,dummy_question1_avg_difference,dummy_question2_mean,dummy_question2_std,dummy_question2_avg_difference,q1_mean,q1_std,q1_avg_difference,q2_mean,q2_std,q2_avg_difference,q3_mean,q3_std,q3_avg_difference,q4_mean,q4_std,q4_avg_difference,q5_mean,q5_std,q5_avg_difference,q6_mean,q6_std,q6_avg_difference,q7_mean,q7_std,q7_avg_difference,q8_mean,q8_std,q8_avg_difference,q9_mean,q9_std,q9_avg_difference,q10_mean,q10_std,q10_avg_difference,conservation_mean,conservation_std,conservation_avg_difference,transcendence_mean,transcendence_std,transcendence_avg_difference,mean_theory,max_theory,min_theory,final_project
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5,2,4366,0,0.0,0.000000,3546,0.812185,17.554455,202,2.775996,37.814332,5726,78.689876,709.200000,1097.958424,105,2648,2543,0.200000,0.309633,0.029611,0.746757,0.717146,12.376541,12.103957,5.526316,33.948718,28.422402,0.200000,0.195595,0.089303,0.548598,0.459295,67.741075,53.227632,21.791169,159.421600,137.630431,0.200000,0.157150,0.064337,0.470679,0.406343,341.632000,308.479400,109.24,836.46,727.22,0.200000,...,7.400000,10.000000,6.123724,7.200000,1.000000,1.224745,1.400000,4.400000,0.894427,1.000000,5.400000,1.949359,2.000000,6.000000,1.224745,1.400000,5.200000,1.303840,1.600000,5.600000,2.073644,2.600000,7.000000,1.224745,1.400000,6.000000,2.000000,2.400000,5.800000,1.483240,1.800000,2.600000,2.701851,3.200000,4.800000,2.167948,2.600000,6.200000,1.303840,1.400000,0.592000,1.213454,1.430000,-1.532000,0.709768,0.794000,0.634000,0.703333,0.573333,0.80
2,5,2,4580,3133,1.0,0.684061,3986,0.870306,9.184332,434,5.685590,29.150447,12230,160.218341,797.200000,538.290535,276,1369,1093,0.200000,0.135045,0.069242,0.343452,0.274210,9.171113,2.048187,6.272727,11.461538,5.188811,0.200000,0.044666,0.136793,0.249949,0.113156,56.055844,60.303227,19.887848,162.167200,142.279352,0.200000,0.215154,0.070957,0.578592,0.507634,764.432000,661.163226,169.82,1529.54,1359.72,0.200000,...,5.000000,16.200000,5.215362,6.400000,1.200000,1.303840,1.600000,4.400000,0.894427,1.000000,4.600000,1.341641,1.600000,6.200000,0.836660,1.000000,5.200000,1.303840,1.600000,4.400000,1.140175,1.400000,5.200000,1.303840,1.600000,5.200000,2.588436,3.000000,6.400000,1.516575,1.800000,3.000000,0.707107,0.800000,5.000000,1.414214,1.600000,6.000000,2.000000,2.400000,1.222000,0.586276,0.720000,-1.260000,0.699893,0.830000,0.701000,0.888333,0.513333,0.85
3,6,3,1888,1888,0.0,1.000000,15

In [ ]:
df.to_excel(out_file)